In [ ]:
%matplotlib ipympl

import glob
import os

import matplotlib.pyplot as plt
import numpy as np

from microhhpy.spatial import Domain, plot_domains
from microhhpy.land import create_land_surface_input, read_corine, plot_corine
from microhhpy.land import get_ifs_vegetation_lut, get_ifs_vegetation_cmap

## Realistic heterogeneous land-use
For realistic cases, we provide methods to create land-surface input from:
- [Corine, Europe, 100m resolution](https://land.copernicus.eu/en/products/corine-land-cover)
- [LCC, Global, 100 m resolution](https://land.copernicus.eu/en/products/global-dynamic-land-cover/copernicus-global-land-service-land-cover-100m-collection-3-epoch-2019-globe)
- [LCC, Global 10 m resolution](https://land.copernicus.eu/en/products/global-dynamic-land-cover/land-cover-2020-raster-10-m-global-annual) (Not yet, TODO!)

In [ ]:
corine_tiff = '/home/scratch1/bart/Corine/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif'
lcc_tiff = '/home/scratch1/bart/LCC/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif'

In [ ]:
"""
Setup domain / projection.
"""
lon = 5.7
lat = 52
proj_str = f'+proj=lcc +lat_1={lat-1} +lat_2={lat+1} +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

dom = Domain(
    xsize = 25600,
    ysize = 25600,
    itot = 256,
    jtot = 256,
    lon = lon,
    lat = lat,
    anchor = 'center',
    proj_str = proj_str)

plot_domains([dom], use_projection=True)

z_soil = np.array([-0.035, -0.175, -0.64 , -1.945])[::-1]

In [ ]:
lu_corine = create_land_surface_input(
    dom.proj.lon,
    dom.proj.lat,
    z_soil,
    land_use_source='corine_100m',
    land_use_tiff=corine_tiff,
    save_binaries=True,
    output_dir='.',
    save_netcdf=True,
    netcdf_file='lsm_input_corine.nc',
)

lu_lcc = create_land_surface_input(
    dom.proj.lon,
    dom.proj.lat,
    z_soil,
    land_use_source='lcc_100m',
    land_use_tiff=lcc_tiff,
    save_binaries=True,
    output_dir='.',
    save_netcdf=True,
    netcdf_file='lsm_input_lcc.nc',
)

In [ ]:
veg_lut = get_ifs_vegetation_lut()
names = veg_lut['name']
ifs_cmap = get_ifs_vegetation_cmap()

plt.figure(figsize=(10, 5), layout='constrained')

ax=plt.subplot(121)
plt.title('Corine 100 m')
plt.pcolormesh(lu_corine.lon, lu_corine.lat, lu_corine.ds.index_veg, vmin=0, vmax=20, cmap=ifs_cmap)

plt.subplot(122, sharex=ax, sharey=ax)
plt.title('LCC 100 m')
plt.pcolormesh(lu_lcc.lon, lu_lcc.lat, lu_lcc.ds.index_veg, vmin=0, vmax=20, cmap=ifs_cmap)

#cbar = plt.colorbar(shrink=0.8)
#cbar.set_ticks(np.arange(21))
#cbar.set_ticklabels([veg_type.replace('_', ' ').title() for veg_type in names])
#cbar.ax.tick_params(labelsize=8)

In [ ]:
# Cleanup!
files = glob.glob('*00*') #+ glob.glob('*.nc')
for f in files:
    os.remove(f)